In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import transformers
import torch

In [3]:
#installing required modules, note:- after installing please restart the runtime, because while training we faced issues that required us to restart after installation
!pip install transformers[torch]
!pip install accelerate -U
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.8 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
from datasets import load_dataset
from transformers import BertTokenizer
import evaluate

#load the pretrained BERT model
pretrained_model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
pretrained_model = AutoModelForMaskedLM.from_pretrained(pretrained_model_name)

#assigning random weights to the loaded model
new_model = AutoModelForMaskedLM.from_pretrained(pretrained_model_name)
for i in new_model.parameters():
  i.data = torch.randn_like(i.data)

model = new_model
#loading the WikiText-2 dataset
data = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")["text"]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model t

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
!pip install transformers
!pip install accelerate -U

In [ ]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer
from datasets import *

#loading the tokenizer for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#loading the data from the input file we generated. The code for generating this input file is in the github repository.
#Please then update the file path according to where you have saved the input file
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path="drive/MyDrive/input.txt", block_size=128)

#preparing the data for the training model with setting mlm to false
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

model = new_model

#defining training arguments
#setting epochs to 5
training_arguments = TrainingArguments(output_dir='./pretrained_bert_scratch', overwrite_output_dir=True, num_train_epochs=5, per_device_train_batch_size=16, save_steps=5000, logging_steps=500)

#training the model giving the training_arguments, data_collator and dataset as input
trainer = Trainer(model=model, args=training_arguments, data_collator=data_collator, train_dataset=dataset)
trainer.train()

Step,Training Loss
500,162.707500
1000,156.136500
1500,151.825300
2000,148.102000
2500,145.816200
3000,143.910800
3500,142.226400
4000,140.773800
4500,139.110400
5000,137.420300


TrainOutput(global_step=28835, training_loss=123.14184408596324, metrics={'train_runtime': 6401.3287, 'train_samples_per_second': 72.064, 'train_steps_per_second': 4.505, 'total_flos': 1.37850600775608e+16, 'train_loss': 123.14184408596324, 'epoch': 5.0})

In [ ]:
#above is the model that we have uploaded to hugging face. We have pushed the model to hugging face with the code shown below:

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1700377689.9627e3e3f992.2267.0:   0%|          | 0.00/13.6k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.60k [00:00<?, ?B/s]

'https://huggingface.co/ayushmodi12/pretrained_bert_scratch/tree/main/'

In [ ]:
#now, below is the code for calculating perplexitiy scores after each epoch. We run it for 4 epochs as we have the model after full 5 epochs already uploaded and so we take the perplexity for 5 epochs from there. We thus have calculated the perplexity score after each of the 5 epochs

In [ ]:
#below is the code for calculating perplexity. we have given the refernce in references section in document

In [3]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForMaskedLM
from datasets import load_dataset
def func(mymodel):
    device = "cuda"
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    model = mymodel.to(device)



    test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
    encodings = tokenizer("\n\n".join(test["text"]),truncation=True, return_tensors="pt").to(device)

    # max_length = model.config.n_positions
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = seq_len
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)

            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            neg_log_likelihood = outputs.loss

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.stack(nlls).mean()
    print(ppl)
    return ppl

In [4]:
from transformers import LineByLineTextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer
from datasets import *

#loading the tokenizer for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#loading the data from the input file we generated. The code for generating this input file is in the github repository.
#Please then update the file path according to where you have saved the input file
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path="drive/MyDrive/input.txt", block_size=128)

#preparing the data for the training model with setting mlm to false
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

model = new_model

#defining training arguments
#setting epochs to 5
training_arguments = TrainingArguments(output_dir='./pretrained_bert_scratch', overwrite_output_dir=True, num_train_epochs=1, per_device_train_batch_size=16, save_steps=5000, logging_steps=500)

#training the model giving the training_arguments, data_collator and dataset as input
trainer = Trainer(model=model, args=training_arguments, data_collator=data_collator, train_dataset=dataset)

#the perp_list which will store the perplexity score after each epoch. We have printed the perplexity scores after each epoch and training loss after every 500 steps
perp_list=[]
for i in range (4):
  trainer.train()
  perp_list.append(func(model))


/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Step,Training Loss
500,142.205700
1000,138.579900
1500,136.052500
2000,134.450000
2500,133.130600
3000,132.214500
3500,131.262900
4000,130.662500
4500,130.317100
5000,129.771300


  0%|          | 0/1 [00:00<?, ?it/s]


tensor(124.1079, device='cuda:0')


Step,Training Loss
500,128.674800
1000,127.332600
1500,126.293200
2000,125.446500
2500,124.731500
3000,123.928700
3500,123.404000
4000,123.105600
4500,122.741300
5000,122.515900


...

  0%|          | 0/1 [00:00<?, ?it/s]


tensor(111.7196, device='cuda:0')


Step,Training Loss
500,115.962300
1000,115.243300
1500,114.134900
2000,113.565400
2500,112.947400
3000,112.575500
3500,111.985800
4000,111.373600
4500,111.426300
5000,110.986900


  0%|          | 0/1 [00:00<?, ?it/s]

tensor(105.7287, device='cuda:0')


In [5]:
print(perp_list)

[tensor(124.1079, device='cuda:0'), tensor(116.5473, device='cuda:0'), tensor(111.7196, device='cuda:0'), tensor(105.7287, device='cuda:0')]


In [6]:
# After 5 epochs we uploaded the model on Hugging Face and pulled out the model to calculate the final perplexity
final_pretrained_model_after_5_epochs=AutoModelForMaskedLM.from_pretrained("ayushmodi12/pretrained_bert_scratch")
perplexity_after_5_epochs=func(final_pretrained_model_after_5_epochs)
print("Final perplexity of the model:")
print(perplexity_after_5_epochs)
perp_list.append(perplexity_after_5_epochs)

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tensor(99.4284, device='cuda:0')
Final perplexity of the model:
tensor(99.4284, device='cuda:0')


In [10]:
#printing the perplexity scores after each of the epoch:
for i in range(1, 6):
  print("Perplexity after Epoch:"+str(i))
  print(perp_list[i-1])

Perplexity after Epoch:1
tensor(124.1079, device='cuda:0')
Perplexity after Epoch:2
tensor(116.5473, device='cuda:0')
Perplexity after Epoch:3
tensor(111.7196, device='cuda:0')
Perplexity after Epoch:4
tensor(105.7287, device='cuda:0')
Perplexity after Epoch:5
tensor(99.4284, device='cuda:0')


In [11]:
#THANK YOU!